In [19]:
# Imports
import BAC0
import pandas as pd

In [2]:
# Init variables
my_ip = '10.187.10.1/24'
bbmdIP = '10.187.43.146:47808'
bbmdTTL = 900

# Create Bacnet Connection
bacnet = BAC0.connect(ip=my_ip, bbmdAddress=bbmdIP, bbmdTTL=bbmdTTL)

2021-07-20 13:19:39,811 - INFO    | Starting BAC0 version 21.02.25 (Complete)
2021-07-20 13:19:39,814 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2021-07-20 13:19:39,815 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2021-07-20 13:19:39,816 - INFO    | Starting TaskManager
2021-07-20 13:19:39,823 - INFO    | Using ip : 10.187.10.1
2021-07-20 13:19:39,825 - INFO    | Starting app...
2021-07-20 13:19:39,826 - INFO    | BAC0 started
2021-07-20 13:19:39,826 - INFO    | Registered as Foreign Device
2021-07-20 13:19:39,828 - INFO    | Update Local COV Task started
2021-07-20 13:19:40,942 - INFO    | Server started : http://10.187.10.1:8111
 * Serving Flask app 'BAC0.web.FlaskServer' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.156:8111/ (Press CTRL+C to quit)


2021-07-20 13:19:41,120 - INFO    | 192.168.140.99 network number is 2
2021-07-20 13:19:43,270 - INFO    | 192.168.140.99 router to [1]
2021-07-20 13:19:44,946 - INFO    | Addr : 192.168.140.99
2021-07-20 13:19:45,094 - INFO    | 192.168.140.99 routing table
2021-07-20 13:19:45,094 - INFO    |     1 2 bytearray(b'')
2021-07-20 13:19:45,095 - INFO    |     2 4 bytearray(b'')
2021-07-20 13:19:46,948 - INFO    | Found those networks : {1, 2}
2021-07-20 13:19:46,949 - INFO    | Discovering network 1
2021-07-20 13:19:49,954 - INFO    | Discovering network 2


In [3]:
devices=bacnet.devices
devices

,Manufacturer,Address,Device ID
Name,,,
AS-P_3658574,Schneider Electric,192.168.140.99,3658574
RP_100001,Schneider Electric,1:192.168.0.11,100001
RP_10002,Schneider Electric,1:192.168.0.12,10002
RP_100003,Schneider Electric,1:192.168.0.13,100003
RP_100004,Schneider Electric,1:192.168.0.14,100004
RP_100005,Schneider Electric,1:192.168.0.15,100005
RP_100006,Schneider Electric,1:192.168.0.16,100006


In [4]:
main_device = devices.iloc[0]
main_device

Manufacturer    Schneider Electric
Address             192.168.140.99
 Device ID                 3658574
Name: AS-P_3658574, dtype: object

In [5]:
controller = BAC0.device(main_device['Address'],main_device[' Device ID'],bacnet)

2021-07-20 13:19:54,213 - INFO    | Changing device state to DeviceDisconnected'>
2021-07-20 13:19:54,503 - INFO    | Changing device state to RPMDeviceConnected'>
2021-07-20 13:19:54,924 - INFO    | Device 3658574:[AS-P_3658574] found... building points list
2021-07-20 13:20:00,670 - INFO    | Ready!
2021-07-20 13:20:00,671 - INFO    | Device defined for normal polling with a delay of 10sec
2021-07-20 13:20:00,673 - INFO    | Polling started, values read every 10 seconds


In [69]:
points=[]
for i in controller.points:
    points.append((i.properties.name,i.properties.type,i.properties.address,i.properties.description))
points=pd.DataFrame(points,columns=['Name','Type','ID','Description'])

In [70]:
points

,Name,Type,ID,Description
0,T Imp FCs A,analogInput,1,
1,T Imp PB P1 T Radiant,analogInput,2,
2,T Imp P2 T Radiant,analogInput,3,
3,T Imp P3 T Radiant,analogInput,4,
4,T Ret PB P1 T Radiant,analogInput,5,
...,...,...,...,...
155,E B11 Plaques Solars,binaryInput,31,
156,E B12 Secundari Solar,binaryInput,32,
157,E B13 Legionela,binaryInput,33,
158,E B14 Secundari ACS,binaryInput,34,


In [71]:
aux = points[['Name']]
aux

,Name
0,T Imp FCs A
1,T Imp PB P1 T Radiant
2,T Imp P2 T Radiant
3,T Imp P3 T Radiant
4,T Ret PB P1 T Radiant
...,...
155,E B11 Plaques Solars
156,E B12 Secundari Solar
157,E B13 Legionela
158,E B14 Secundari ACS


In [80]:
csv = pd.read_csv('../input/flix.csv')
csv = csv.rename(columns={'NAME':'Name'})

In [86]:
merged = pd.merge(csv,points)

In [87]:
merged

,TYPE,Name,DESCR,Value,Type,ID,Description
0,bacnet.point.analog.Value,Consigna Clima COB Passad Curt,NaN,"30,00",analogValue,37,
1,bacnet.point.analog.Value,Consigna Clima COB Passad Llarg,NaN,"30,00",analogValue,38,
2,bacnet.point.analog.Value,Consigna Clima P1 Menjador,NaN,"23,00",analogValue,39,
3,bacnet.point.analog.Value,Consigna Clima P1 Passad Curt,NaN,"25,00",analogValue,40,
4,bacnet.point.analog.Value,Consigna Clima P1 Passad Llarg,NaN,"25,00",analogValue,42,
...,...,...,...,...,...,...,...
140,bacnet.point.analog.Value,Velocitat Ventilador Clima P3 Menjador,NaN,"33,00",analogValue,81,
141,bacnet.point.analog.Value,Velocitat Ventilador Clima P3 Passad Curt,NaN,"0,00",analogValue,82,
142,bacnet.point.analog.Value,Velocitat Ventilador Clima P3 Passad Llarg,NaN,"0,00",analogValue,83,
143,bacnet.point.analog.Value,Velocitat Ventilador Clima P3 Sala Estar,NaN,"33,00",analogValue,84,


In [91]:
aux2 = csv[['Name']]

In [92]:
pd.concat([aux,aux2]).drop_duplicates(keep=False)

,Name
0,T Imp FCs A
15,T Imp FCs B
16,T Imp FCs C
47,Reg Valv Fancoils Circuit A
48,Reg Valv Fancoils Circuit B
49,Reg Valv Fancoils Circuit C
50,Consigna Temp Impulsio Fancoils Fred
51,Consigna Temp Impulsio Fancoils Calor
137,Temperatura Calculada Corba Baixa_copy
138,Temperatura Calculada Corba Baixa_copy_copy


In [97]:
points[points['Name'].str.contains('Temperatura Calculada')]

,Name,Type,ID,Description
136,Temperatura Calculada Corba Baixa,analogValue,92,
137,Temperatura Calculada Corba Baixa_copy,analogValue,93,
138,Temperatura Calculada Corba Baixa_copy_copy,analogValue,94,


In [98]:
controller['Consigna Clima COB Passad Curt'].history

2021-07-20 13:20:00.063699    8.0
2021-07-20 13:20:18.082382    8.0
2021-07-20 13:20:29.196703    8.0
2021-07-20 13:20:40.540308    8.0
2021-07-20 13:20:51.666411    8.0
                             ... 
2021-07-20 16:06:53.669677    8.0
2021-07-20 16:07:05.456417    8.0
2021-07-20 16:07:17.389078    8.0
2021-07-20 16:07:29.257133    8.0
2021-07-20 16:07:41.016240    8.0
Name: AS-P_3658574/Consigna Clima COB Passad Curt, Length: 856, dtype: float64